In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 50)

import os
# Use this to see how much memory the dataframes use
from sys import getsizeof

from vivarium_research_prl.utils import sizemb
from vivarium_research_prl import datatypes
# from vivarium_research_prl.noise import corruption, fake_names, noisify
# from vivarium_research_prl.find_kids import datasets, noisify_data

!date
!whoami
!uname -a
!pwd

Tue 21 Mar 2023 01:56:16 PM PDT
ndbs
Linux int-slurm-sarchive-p0011 5.4.0-135-generic #152-Ubuntu SMP Wed Nov 23 20:19:22 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/code/ndbs/vivarium_research_prl/model_validation


In [3]:
import os

# Find data

Looks like the same run we were looking at before, but post-processing has been fixed to keep year of observation in the data.

```
/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/vv_tax_observers/united_states_of_america/2023_03_06_12_42_55/final_results/2023_03_14_16_41_22
```

In [2]:
project_output_dir = \
    '/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop'
output_subdir = (
    'results/vv_tax_observers'
    '/united_states_of_america/2023_03_06_12_42_55'
    '/final_results/2023_03_14_16_41_22'
)
output_dir = f'{project_output_dir}/{output_subdir}'

!ls -halt $output_dir

total 14G
drwxrwsr-x  2 albrja IHME-Simulationscience 4.0K Mar 15 10:01 .
-rw-r--r--  1 albrja IHME-Simulationscience  59M Mar 15 10:01 wic_observer.csv.bz2
-rw-r--r--  1 albrja IHME-Simulationscience 1.5G Mar 15 10:01 tax_dependents_observer.csv.bz2
-rw-r--r--  1 albrja IHME-Simulationscience 6.8G Mar 15 09:57 tax_w2_observer.csv.bz2
-rw-r--r--  1 albrja IHME-Simulationscience 4.1G Mar 15 09:56 tax_1040_observer.csv.bz2
-rw-r--r--  1 albrja IHME-Simulationscience 192M Mar 15 09:48 social_security_observer.csv.bz2
-rw-r--r--  1 albrja IHME-Simulationscience  35M Mar 15 09:36 household_survey_observer_cps.csv.bz2
-rw-r--r--  1 albrja IHME-Simulationscience 6.7M Mar 15 09:36 household_survey_observer_acs.csv.bz2
-rw-r--r--  1 albrja IHME-Simulationscience 638M Mar 15 09:36 decennial_census_observer.csv.bz2
drwxrwsr-x 12 albrja IHME-Simulationscience 5.5K Mar 14 20:25 ..


# Use `os.scandir` to open first 100 rows of all files in directory for inspection


https://docs.python.org/3/library/os.html#os.scandir

In [36]:
%%time
observer_samples = {}
nrows = 100
ext = '.csv.bz2'
with os.scandir(output_dir) as directory_contents:
    for entry in directory_contents:
        if entry.name.endswith(ext):
            basename = entry.name[:-len(ext)]
            observer_samples[basename] = pd.read_csv(entry, nrows=nrows)
observer_samples.keys()

CPU times: user 180 ms, sys: 15.5 ms, total: 195 ms
Wall time: 197 ms


dict_keys(['household_survey_observer_cps', 'wic_observer', 'household_survey_observer_acs', 'social_security_observer', 'tax_w2_observer', 'tax_1040_observer', 'decennial_census_observer', 'tax_dependents_observer'])

In [29]:
# Darn, the dict doesn't add up the sizes of its stored keys or values
getsizeof(observer_samples)

360

In [32]:
# 566 bytes for the dict's keys
sum(getsizeof(basename) for basename in observer_samples.keys())

566

In [31]:
# 735 KB for the sample dataframes
sum(getsizeof(df) for df in observer_samples.values())

735302

In [11]:
observer_samples['household_survey_observer_cps']

,middle_initial,household_id,zipcode,housing_type,simulant_id,guardian_1,date_of_birth,last_name,first_name,age,street_name,city,guardian_2,sex,survey_date,unit_number,street_number,state
0,J,2284_653,10703.0,Standard,2284_1646,2284_-1,1943-12-27,Nurse,Barbara,76.262593,morning gln dr,big flts,2284_-1,Female,2020-04-29,NaN,3741.0,NY
1,M,2284_653,10703.0,Standard,2284_1647,2284_-1,1964-03-21,Nurse,Jean,56.029565,morning gln dr,big flts,2284_-1,Female,2020-04-29,NaN,3741.0,NY
2,K,2284_1573,43232.0,Standard,2284_3966,2284_-1,1957-10-18,Tollefsrud,Lynn,62.451908,delridge way sw,lake,2284_-1,Female,2020-04-29,NaN,108.0,OH
3,C,2284_3260,19061.0,Standard,2284_8141,2284_-1,1953-09-19,Winter,Dianne,66.532583,charm tree ct,shrewsbury,2284_-1,Female,2020-04-29,NaN,19420.0,PA
4,J,2284_3260,19061.0,Standard,2284_8142,2284_-1,1950-09-11,Winter,Robert,69.555260,charm tree ct,shrewsbury,2284_-1,Male,2020-04-29,NaN,19420.0,PA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,R,2284_50054,92833.0,Standard,2284_124350,2284_-1,1990-10-07,Vanderark,Evan,29.482749,green bay,brentwood,2284_-1,Male,2020-04-29,NaN,4827.0,CA
96,K,2284_50054,92833.0,Standard,2284_124351,2284_124349,2018-05-01,Vanderark,Izabella,1.918297,green bay,brentwood,2284_124350,Female,2020-04-29,NaN,4827.0,CA
97,T,2284_50133,91732.0,Standard,2284_124526,2284_-1,1968-10-14,Gonzalez,Kelly,51.463065,s clifton park avenue,beverly hills,2284_-1,Female,2020-04-29,NaN,382.0,CA
98,J,2284_50133,91732.0,Standard,2284_124527,2284_-1,1966-03-23,Gonzalez,Kurt,54.025843,s clifton park avenue,beverly hills,2284_-1,Male,2020-04-29,NaN,382.0,CA


In [12]:
observer_samples['household_survey_observer_cps'].dtypes

middle_initial     object
household_id       object
zipcode           float64
housing_type       object
simulant_id        object
guardian_1         object
date_of_birth      object
last_name          object
first_name         object
age               float64
street_name        object
city               object
guardian_2         object
sex                object
survey_date        object
unit_number        object
street_number     float64
state              object
dtype: object

# Create a dictionary storing dtype Series for all dataframes, and concatenate all dtype Series into a single dataframe

In [26]:
dtypes_dict = {filename: df.dtypes for filename, df in observer_samples.items()}

dtypes_df = (
    pd.concat(dtypes_dict, axis=1)
    .rename_axis(index='field', columns='observer')
)
dtypes_df

observer,household_survey_observer_cps,wic_observer,household_survey_observer_acs,social_security_observer,tax_w2_observer,tax_1040_observer,decennial_census_observer,tax_dependents_observer
field,,,,,,,,
middle_initial,object,object,object,object,object,object,object,object
household_id,object,object,object,NaN,NaN,NaN,NaN,NaN
zipcode,float64,float64,float64,NaN,NaN,NaN,float64,NaN
housing_type,object,object,object,NaN,NaN,object,object,object
simulant_id,object,object,object,object,object,object,object,object
guardian_1,object,object,object,NaN,NaN,NaN,object,NaN
date_of_birth,object,object,object,object,object,object,object,object
last_name,object,object,object,object,object,object,object,object
first_name,object,object,object,object,object,object,object,object


# Check which fields are in all observers

In [28]:
[field for field in dtypes_df.index if dtypes_df.loc[field].notna().all()]

['middle_initial', 'simulant_id', 'date_of_birth', 'last_name', 'first_name']

# Check which fields are in exactly one observer

In [41]:
temp = [field for field in dtypes_df.index if dtypes_df.loc[field].notna().sum()==1]
temp

['wic_year',
 'event_type',
 'event_date',
 'employer_street_number',
 'employer_id',
 'is_w2',
 'employer_name',
 'employer_street_name',
 'employer_city',
 'employer_zipcode',
 'employer_state',
 'income',
 'employer_unit_number',
 'joint_filer',
 'itin',
 'census_year',
 'guardian_id']

In [44]:
dtypes_df.loc['wic_year', 'wic_observer']

dtype('int64')

In [51]:
np.isnan(dtypes_df.loc['wic_year', 'tax_w2_observer'])

True

In [53]:
# Doesn't work...
{field: col for field in temp for col in dtypes_df
 if dtypes_df.loc[field,col] != dtypes_df.loc[field,col]} 

{'wic_year': 'tax_dependents_observer',
 'event_type': 'tax_dependents_observer',
 'event_date': 'tax_dependents_observer',
 'employer_street_number': 'tax_dependents_observer',
 'employer_id': 'tax_dependents_observer',
 'is_w2': 'tax_dependents_observer',
 'employer_name': 'tax_dependents_observer',
 'employer_street_name': 'tax_dependents_observer',
 'employer_city': 'tax_dependents_observer',
 'employer_zipcode': 'tax_dependents_observer',
 'employer_state': 'tax_dependents_observer',
 'income': 'tax_dependents_observer',
 'employer_unit_number': 'tax_dependents_observer',
 'joint_filer': 'tax_dependents_observer',
 'itin': 'tax_dependents_observer',
 'census_year': 'tax_dependents_observer',
 'guardian_id': 'decennial_census_observer'}

# See how many observers each field appears in

In [57]:
dtypes_df.notna().sum(axis=1).sort_values()

field
event_type                       1
itin                             1
joint_filer                      1
employer_unit_number             1
income                           1
employer_state                   1
employer_zipcode                 1
employer_city                    1
employer_street_name             1
employer_name                    1
is_w2                            1
employer_id                      1
employer_street_number           1
event_date                       1
census_year                      1
wic_year                         1
guardian_id                      1
race_ethnicity                   2
relation_to_household_head       2
survey_date                      2
mailing_address_state            3
mailing_address_po_box           3
tax_year                         3
mailing_address_street_number    3
mailing_address_city             3
mailing_address_street_name      3
mailing_address_zipcode          3
mailing_address_unit_number      3
household_id  

In [81]:
dtypes_df.loc['race_ethnicity', :].notna()

observer
household_survey_observer_cps    False
wic_observer                      True
household_survey_observer_acs    False
social_security_observer         False
tax_w2_observer                  False
tax_1040_observer                False
decennial_census_observer         True
tax_dependents_observer          False
Name: race_ethnicity, dtype: bool

# Make a simple convenience wrapper class to enable accessing dictionary items using tab completion instead of having to fully type out or copy/paste the keys

In [39]:
class Convenience:
    def __init__(self, mapping):
        self.__dict__ = dict(mapping)
        
    def asdict():
        return dict(self.__dict__)

samples = Convenience(observer_samples)
dtypes = Convenience(dtypes_dict)
dir(dtypes)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'asdict',
 'decennial_census_observer',
 'household_survey_observer_acs',
 'household_survey_observer_cps',
 'social_security_observer',
 'tax_1040_observer',
 'tax_dependents_observer',
 'tax_w2_observer',
 'wic_observer']

In [19]:
dtypes.household_survey_observer_acs

middle_initial     object
household_id       object
zipcode           float64
housing_type       object
simulant_id        object
guardian_1         object
date_of_birth      object
last_name          object
first_name         object
age               float64
street_name        object
city               object
guardian_2         object
sex                object
survey_date        object
unit_number        object
street_number       int64
state              object
dtype: object

In [34]:
getsizeof(samples), getsizeof(dtypes)

(48, 48)

# Look at SSA data

Expected columns from [Social Security Observer
 documentation](https://vivarium-research.readthedocs.io/en/latest/models/concept_models/vivarium_census_synthdata/concept_model.html#social-security-observer):

```
Unique simulant ID (for PRL tracking)

First name

Middle initial

Last name

DOB (stored as a string in YYYYMMDD format)

Social Security Number

Type of event

Date of event (stored as a string in YYYYMMDD format)
```

## Questions

1. Rowan's SSN card has his full middle name -- should we do this instead of middle initial?
1. Rowan's SSN card has a date in the format MM/DD/YYYY -- should we use this format instead, or not?
1. Rowan's SSN card is dated 10 days after his birthday -- should we add a time lag between DOB or immigration date and creation date? (We have added a ticket for this)

In [20]:
dtypes.social_security_observer

ssn               object
middle_initial    object
event_type        object
last_name         object
first_name        object
event_date        object
simulant_id       object
date_of_birth     object
dtype: object

In [22]:
samples.social_security_observer

,ssn,middle_initial,event_type,last_name,first_name,event_date,simulant_id,date_of_birth
0,786-77-6454,M,creation,Bingham,Peter,1921-05-13,2284_733465,1921-05-13
1,688-88-6377,D,creation,Bingham,Janice,1921-06-21,2284_733466,1921-06-21
2,651-33-9561,H,creation,Burdick,Anna,1921-07-13,2284_776885,1921-07-13
3,665-25-7858,M,creation,Tapia,Gertrude,1921-07-26,2284_869365,1921-07-26
4,875-10-2359,R,creation,Feucht,Mildred,1921-10-01,2284_13652,1921-10-01
...,...,...,...,...,...,...,...,...
95,766-54-7928,V,creation,George,Rosie,1923-01-29,2284_558770,1923-01-29
96,163-22-8752,J,creation,Smith,Katherine,1923-01-30,2284_327101,1923-01-30
97,659-21-2287,B,creation,Doyle,Sally,1923-01-30,2284_434193,1923-01-30
98,185-67-0764,K,creation,Nyburg,Hilda,1923-02-02,2284_756151,1923-02-02


In [82]:
%%time
df_ss = pd.read_csv(
    f'{output_dir}/social_security_observer.csv.bz2',
    nrows=1_000_000,
)
df_ss

CPU times: user 3.75 s, sys: 97.2 ms, total: 3.85 s
Wall time: 3.89 s


,ssn,middle_initial,event_type,last_name,first_name,event_date,simulant_id,date_of_birth
0,786-77-6454,M,creation,Bingham,Peter,1921-05-13,2284_733465,1921-05-13
1,688-88-6377,D,creation,Bingham,Janice,1921-06-21,2284_733466,1921-06-21
2,651-33-9561,H,creation,Burdick,Anna,1921-07-13,2284_776885,1921-07-13
3,665-25-7858,M,creation,Tapia,Gertrude,1921-07-26,2284_869365,1921-07-26
4,875-10-2359,R,creation,Feucht,Mildred,1921-10-01,2284_13652,1921-10-01
...,...,...,...,...,...,...,...,...
999995,461-43-6722,P,death,Hindes,Frances,2021-08-18,2284_645448,1948-12-02
999996,378-10-3757,C,death,Rhodes,Karen,2021-08-18,2284_499762,1948-12-10
999997,847-72-9612,S,death,Ayala-Rivera,Thomas,2021-08-18,2284_124489,1949-01-05
999998,811-81-0649,S,death,Horton,Ronald,2021-08-18,2284_165020,1949-02-14


In [71]:
%%time
df_ss = datatypes.load_csv_data(
    f'{output_dir}/social_security_observer.csv.bz2',
    convert_str_ids=True,
    nrows=1_000_000,
)
df_ss

CPU times: user 7.12 s, sys: 244 ms, total: 7.37 s
Wall time: 7.38 s


,ssn,middle_initial,event_type,last_name,first_name,event_date,simulant_id,date_of_birth
0,786776454,M,creation,Bingham,Peter,1921-05-13,2284000733465,1921-05-13
1,688886377,D,creation,Bingham,Janice,1921-06-21,2284000733466,1921-06-21
2,651339561,H,creation,Burdick,Anna,1921-07-13,2284000776885,1921-07-13
3,665257858,M,creation,Tapia,Gertrude,1921-07-26,2284000869365,1921-07-26
4,875102359,R,creation,Feucht,Mildred,1921-10-01,2284000013652,1921-10-01
...,...,...,...,...,...,...,...,...
999995,461436722,P,death,Hindes,Frances,2021-08-18,2284000645448,1948-12-02
999996,378103757,C,death,Rhodes,Karen,2021-08-18,2284000499762,1948-12-10
999997,847729612,S,death,Ayala-Rivera,Thomas,2021-08-18,2284000124489,1949-01-05
999998,811810649,S,death,Horton,Ronald,2021-08-18,2284000165020,1949-02-14


In [64]:
df_ss.dtypes

ssn                  int32
middle_initial    category
event_type          object
last_name         category
first_name        category
event_date          object
simulant_id          int64
date_of_birth     category
dtype: object

In [72]:
creation_only = df_ss.loc[df_ss.event_type == 'creation']
creation_only

,ssn,middle_initial,event_type,last_name,first_name,event_date,simulant_id,date_of_birth
0,786776454,M,creation,Bingham,Peter,1921-05-13,2284000733465,1921-05-13
1,688886377,D,creation,Bingham,Janice,1921-06-21,2284000733466,1921-06-21
2,651339561,H,creation,Burdick,Anna,1921-07-13,2284000776885,1921-07-13
3,665257858,M,creation,Tapia,Gertrude,1921-07-26,2284000869365,1921-07-26
4,875102359,R,creation,Feucht,Mildred,1921-10-01,2284000013652,1921-10-01
...,...,...,...,...,...,...,...,...
999872,101886992,R,creation,Sultana,Duane,2021-08-18,2284001025149,1941-12-08
999894,495496328,S,creation,Sawyer,Patsy,2021-08-18,2284001025175,1943-03-12
999923,106773116,A,creation,Rigney,Claire,2021-08-18,2284001024982,1944-08-07
999934,187941864,B,creation,Busch,Shirley,2021-08-18,2284001025105,1945-03-15


In [73]:
creation_only.loc[creation_only.event_date == creation_only.date_of_birth]

,ssn,middle_initial,event_type,last_name,first_name,event_date,simulant_id,date_of_birth
0,786776454,M,creation,Bingham,Peter,1921-05-13,2284000733465,1921-05-13
1,688886377,D,creation,Bingham,Janice,1921-06-21,2284000733466,1921-06-21
2,651339561,H,creation,Burdick,Anna,1921-07-13,2284000776885,1921-07-13
3,665257858,M,creation,Tapia,Gertrude,1921-07-26,2284000869365,1921-07-26
4,875102359,R,creation,Feucht,Mildred,1921-10-01,2284000013652,1921-10-01
...,...,...,...,...,...,...,...,...
998689,418065119,F,creation,Martin,Jacob,2021-07-21,2284001023446,2021-07-21
998690,111583661,J,creation,Fletcher,Moshe,2021-07-21,2284001023454,2021-07-21
998691,530393088,L,creation,Feehan,Kaylee,2021-07-21,2284001023483,2021-07-21
998692,304290148,M,creation,Arora,Zoe,2021-07-21,2284001023566,2021-07-21


In [79]:
creation_only.loc[creation_only.event_date != creation_only.date_of_birth]

,ssn,middle_initial,event_type,last_name,first_name,event_date,simulant_id,date_of_birth
966529,63623526,P,creation,Griffin,William,2020-04-29,2284001002664,1926-06-13
966636,518625594,A,creation,Tapscott,Dolores,2020-04-29,2284001002533,1933-04-20
966656,208491946,H,creation,Martin,Joyce,2020-04-29,2284001002661,1934-03-02
966659,323758179,J,creation,Griffin,David,2020-04-29,2284001002663,1934-03-22
966666,588502013,L,creation,Smith,Bob,2020-04-29,2284001002511,1934-08-19
...,...,...,...,...,...,...,...,...
999872,101886992,R,creation,Sultana,Duane,2021-08-18,2284001025149,1941-12-08
999894,495496328,S,creation,Sawyer,Patsy,2021-08-18,2284001025175,1943-03-12
999923,106773116,A,creation,Rigney,Claire,2021-08-18,2284001024982,1944-08-07
999934,187941864,B,creation,Busch,Shirley,2021-08-18,2284001025105,1945-03-15


In [74]:
df_ss.loc[df_ss.event_date == df_ss.date_of_birth]

,ssn,middle_initial,event_type,last_name,first_name,event_date,simulant_id,date_of_birth
0,786776454,M,creation,Bingham,Peter,1921-05-13,2284000733465,1921-05-13
1,688886377,D,creation,Bingham,Janice,1921-06-21,2284000733466,1921-06-21
2,651339561,H,creation,Burdick,Anna,1921-07-13,2284000776885,1921-07-13
3,665257858,M,creation,Tapia,Gertrude,1921-07-26,2284000869365,1921-07-26
4,875102359,R,creation,Feucht,Mildred,1921-10-01,2284000013652,1921-10-01
...,...,...,...,...,...,...,...,...
998689,418065119,F,creation,Martin,Jacob,2021-07-21,2284001023446,2021-07-21
998690,111583661,J,creation,Fletcher,Moshe,2021-07-21,2284001023454,2021-07-21
998691,530393088,L,creation,Feehan,Kaylee,2021-07-21,2284001023483,2021-07-21
998692,304290148,M,creation,Arora,Zoe,2021-07-21,2284001023566,2021-07-21


In [75]:
df_ss.event_type.unique()

array(['creation', 'death'], dtype=object)

In [77]:
df_ss.event_type.value_counts()

creation    987886
death        12114
Name: event_type, dtype: int64

In [80]:
df_ss.loc[df_ss.event_date.astype('datetime64') == df_ss.date_of_birth.astype('datetime64')]

,ssn,middle_initial,event_type,last_name,first_name,event_date,simulant_id,date_of_birth
0,786776454,M,creation,Bingham,Peter,1921-05-13,2284000733465,1921-05-13
1,688886377,D,creation,Bingham,Janice,1921-06-21,2284000733466,1921-06-21
2,651339561,H,creation,Burdick,Anna,1921-07-13,2284000776885,1921-07-13
3,665257858,M,creation,Tapia,Gertrude,1921-07-26,2284000869365,1921-07-26
4,875102359,R,creation,Feucht,Mildred,1921-10-01,2284000013652,1921-10-01
...,...,...,...,...,...,...,...,...
998689,418065119,F,creation,Martin,Jacob,2021-07-21,2284001023446,2021-07-21
998690,111583661,J,creation,Fletcher,Moshe,2021-07-21,2284001023454,2021-07-21
998691,530393088,L,creation,Feehan,Kaylee,2021-07-21,2284001023483,2021-07-21
998692,304290148,M,creation,Arora,Zoe,2021-07-21,2284001023566,2021-07-21


In [76]:
!conda list

# packages in environment at /ihme/homes/ndbs/miniconda3/envs/prlpy38:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       2_gnu    conda-forge
altair                    4.2.0                    pypi_0    pypi
anyio                     3.6.2                    pypi_0    pypi
argon2-cffi               21.3.0                   pypi_0    pypi
argon2-cffi-bindings      21.2.0                   pypi_0    pypi
asttokens                 2.0.8                    pypi_0    pypi
attrs                     22.1.0                   pypi_0    pypi
babel                     2.10.3                   pypi_0    pypi
backcall                  0.2.0                    pypi_0    pypi
beautifulsoup4            4.11.1                   pypi_0    pypi
bleach                    5.0.1                    pypi_0    pypi
blosc                     1.21.1               h83bc5f